In [1]:
import os, random, numpy as np, tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_fscore_support
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

In [ ]:
# 1) Load CIFAR-10
(num_classes, input_shape) = (10, (32, 32, 3))
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
y_train = y_train.flatten()
y_test  = y_test.flatten()

In [ ]:
# train/val split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.1, random_state=SEED, stratify=y_train
)

In [ ]:
# Normalize to [0,1]
x_train = x_train.astype("float32")/255.0
x_val   = x_val.astype("float32")/255.0
x_test  = x_test.astype("float32")/255.0

In [ ]:
# 2) Build a clean CNN from scratch
def make_scratch_cnn(input_shape=(32,32,3), num_classes=10):
    inputs = keras.Input(shape=input_shape)
    x = inputs

    # Block 1
    x = layers.Conv2D(32, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(32, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)

    # Block 2
    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(64, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)

    # Block 3
    x = layers.Conv2D(128, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(128, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.25)(x)

    x = layers.Flatten()(x)
    x = layers.Dense(256)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(0.5)(x)

    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs, name="scratch_cnn")

In [ ]:
model_scratch = make_scratch_cnn(input_shape, num_classes)
model_scratch.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model_scratch.summary()

In [ ]:
# 3) Train
early = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy")
history = model_scratch.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=30,
    batch_size=128,
    callbacks=[early],
    verbose=2
)

In [ ]:
# 4) Evaluate + Metrics
test_probs = model_scratch.predict(x_test, batch_size=256, verbose=0)
y_pred = test_probs.argmax(axis=1)

acc = (y_pred == y_test).mean()
print(f"[Scratch CNN] Test Accuracy: {acc:.4f}")

print("\nClassification report (per-class precision/recall/F1):")
print(classification_report(y_test, y_pred, digits=4))

cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)

In [ ]:
# Macro/micro precision, recall, f1
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='macro')
print(f"Macro Precision: {prec:.4f} | Macro Recall: {rec:.4f} | Macro F1: {f1:.4f}")
prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='micro')
print(f"Micro Precision: {prec:.4f} | Micro Recall: {rec:.4f} | Micro F1: {f1:.4f}")

In [ ]:
# ROC-AUC (one-vs-rest, needs probabilities)
# Create one-hot labels
y_test_oh = keras.utils.to_categorical(y_test, num_classes)
try:
    auc_ovr = roc_auc_score(y_test_oh, test_probs, average="macro", multi_class="ovr")
    auc_ovo = roc_auc_score(y_test_oh, test_probs, average="macro", multi_class="ovo")
    print(f"ROC-AUC (macro, OVR): {auc_ovr:.4f} | ROC-AUC (macro, OVO): {auc_ovo:.4f}")
except Exception as e:
    print("ROC-AUC could not be computed:", e)